In [47]:
import atom
import timeit
import numba, numpy as np
from typing import NamedTuple, Tuple
from enum import Enum
from dataclasses import dataclass
Pos = NamedTuple('Pos', [('row', int), ('col', int)])

@dataclass
class Location:
    row: int
    col: int

@dataclass(frozen=True)
class Location1:
    row: int
    col: int

class Location2:
    def __init__(self, a, b) -> None:
        self[0]: int = a
        self[1]: int = b



In [48]:
def run_test(f, num, *args):
  return timeit.timeit(lambda: f(*args), number=num, globals={'atom': atom})

def print_results(results: list[tuple[str,int]]):
    # Assuming results is a list of tuples (name, time)
    base_name, base_time = results[0]
    print(f"{base_name}: {base_time:.4f} seconds (base)")
    for name, time in results[1:]:
        ratio = base_time / time
        print(f"{name}: {time:.4f} seconds, Speed relative to {base_name}: {ratio:.2f}x")

In [80]:
def make_tuple(a,b): return (a,b)
def make_arr(a,b): return [a,b]
def make_dir(a,b): return a,b
def make_pos(a,b): return Pos(a,b)
def make_loc(a,b): return Location(a,b)
def make_loc1(a,b): return Location1(a,b)
def make_cls(a,b): return Location2(a,b)
def make_atom(a,b): return (b << 16) | a

num = 10_000_000
a, b = 25, 25
result = []
result.append(("tup", run_test(make_tuple, num, a, b)))
result.append(("atm", run_test(make_atom, num, a, b)))
result.append(("arr", run_test(make_arr, num, a, b)))
result.append(("dir", run_test(make_dir, num, a, b)))
result.append(("pos", run_test(make_pos, num, a, b)))
result.append(("loc", run_test(make_loc, num, a, b)))
result.append(("loc", run_test(make_loc1, num, a, b)))
result.append(("cls", run_test(make_cls, num, a, b)))
print_results(result)

tup: 2.8735 seconds (base)
atm: 3.7721 seconds, Speed relative to tup: 0.76x
arr: 2.9979 seconds, Speed relative to tup: 0.96x
dir: 2.8252 seconds, Speed relative to tup: 1.02x
pos: 8.2412 seconds, Speed relative to tup: 0.35x
loc: 6.3131 seconds, Speed relative to tup: 0.46x
loc: 12.0672 seconds, Speed relative to tup: 0.24x
cls: 5.5942 seconds, Speed relative to tup: 0.51x


In [81]:
def get_row_tuple(val): return val[0]
def get_row_arr(val): return val[0]
def get_row_dir(val): return val[0]
def get_row_pos(val): return val[0]
def get_row_loc(val): return val[0]
def get_row_cls(val): return val[0]
def get_row_atom(val): return val & 0xFFFF
def get_col_atom(val): return val >> 16 & 0xFFFF
def get_bth_atom(val): return val & 0xFFFF, val >> 16 & 0xFFFF

num = 10_000_000
a, b = 25, 25
t1 = (a,b)
t2 = [a,b]
t4 = Pos(a,b)
t5 = Location(a,b)
t5_1 = Location1(a,b)
t6 = Location2(a,b)
t7 = (b << 16) | a

result = []
result.append(("tup    ", run_test(get_row_tuple, num, t1)))
result.append(("atm_row", run_test(get_row_atom, num, t7)))
result.append(("atm_col", run_test(get_col_atom, num, t7)))
result.append(("atm_col", run_test(get_bth_atom, num, t7)))
result.append(("arr    ", run_test(get_row_arr, num, t2)))
# result.append(("dir", run_test(get_row_dir, num, a, b)))
result.append(("pos    ", run_test(get_row_pos, num, t4)))
result.append(("loc    ", run_test(get_row_loc, num, t5)))
result.append(("loc    ", run_test(get_row_loc, num, t5_1)))
result.append(("cls    ", run_test(get_row_cls, num, t6)))
print_results(result)

tup    : 2.2600 seconds (base)
atm_row: 2.2080 seconds, Speed relative to tup    : 1.02x
atm_col: 2.8219 seconds, Speed relative to tup    : 0.80x
atm_col: 3.7596 seconds, Speed relative to tup    : 0.60x
arr    : 2.3571 seconds, Speed relative to tup    : 0.96x
pos    : 2.4126 seconds, Speed relative to tup    : 0.94x
loc    : 2.7370 seconds, Speed relative to tup    : 0.83x
loc    : 2.7637 seconds, Speed relative to tup    : 0.82x
cls    : 2.2348 seconds, Speed relative to tup    : 1.01x


In [43]:
def test_Values(a,b): return a == b
def test_atm_row(a,b): return bool(a & b & 0xFFFF)
def test_atm_col1(a,b): return bool(((a & b) >> 16) & 0xFFFF)
def test_atm_col2(a,b): return bool((a & b) & 0xFFFF0000)

num = 10_000_000
a, b = 25, 20
t1, t1_1, t1_2, t1_3 = (a,b), (a,b), (b,a), (a+10,b+10)
t2, t2_1, t2_2, t2_3 = [a,b], [a,b], [b,a], [a+10,b+10]
t4, t4_1, t4_2, t4_3 = Pos(a,b), Pos(a,b), Pos(b,a), Pos(a+10,b+10)
t5, t5_1, t5_2, t5_3 = Location(a,b), Location(a,b), Location(b,a), Location(a+10,b+10)
t6, t6_1, t6_2, t6_3 = Location2(a,b), Location2(a,b), Location2(b,a), Location2(a+10,b+10)
t7, t7_1, t7_2, t7_3 = (b << 16) | a, (b << 16) | a, (a << 16) | b, ((b+10) << 16) | (a+10)

result = []
result.append((f"tup    {t1==t1_1, t1==t1_2, t1==t1_3}", run_test(test_Values, num, t1, t1_1)))
result.append((f"atm_row{t7==t7_1, t7==t7_2, t7==t7_3}", run_test(test_atm_row, num, t7, t7_1)))
result.append((f"atm_col{t7==t7_1, t7==t7_2, t7==t7_3}", run_test(test_atm_col1, num, t7, t7_1)))
result.append((f"atm_col{t7==t7_1, t7==t7_2, t7==t7_3}", run_test(test_atm_col2, num, t7, t7_1)))
result.append((f"atm_bth{t7==t7_1, t7==t7_2, t7==t7_3}", run_test(test_Values, num, t7, t7_1)))
result.append((f"arr    {t2==t2_1, t2==t2_2, t2==t2_3}", run_test(test_Values, num, t2, t2_1)))
# result.append(("dir", run_test(test_Values, num, a, b)))
result.append((f"pos    {t4==t4_1, t4==t4_2, t4==t4_3}", run_test(test_Values, num, t4, t4_1)))
result.append((f"loc    {t5==t5_1, t5==t5_2, t5==t5_3}", run_test(test_Values, num, t5, t5_1)))
result.append((f"cls    {t6==t6_1, t6==t6_2, t6==t6_3}", run_test(test_Values, num, t6, t6_1)))
print_results(result)

tup    (True, False, False): 3.4639 seconds (base)
atm_row(True, False, False): 4.4761 seconds, Speed relative to tup    (True, False, False): 0.77x
atm_col(True, False, False): 4.3468 seconds, Speed relative to tup    (True, False, False): 0.80x
atm_col(True, False, False): 4.3253 seconds, Speed relative to tup    (True, False, False): 0.80x
atm_bth(True, False, False): 2.2867 seconds, Speed relative to tup    (True, False, False): 1.51x
arr    (True, False, False): 2.6002 seconds, Speed relative to tup    (True, False, False): 1.33x
pos    (True, False, False): 2.6860 seconds, Speed relative to tup    (True, False, False): 1.29x
loc    (True, False, False): 5.4345 seconds, Speed relative to tup    (True, False, False): 0.64x
cls    (False, False, False): 2.2838 seconds, Speed relative to tup    (True, False, False): 1.52x


In [52]:
@numba.njit
def make_tuple2(a,b): return (a,b)
@numba.njit
def make_arr2(a,b): return [a,b]
@numba.njit
def make_arr3(a,b): return np.array([a,b])
@numba.njit
def make_dir2(a,b): return a,b
@numba.njit
def make_pos2(a,b): return Pos(a,b)
# @numba.njit
# def make_loc2(a,b): return Location(a,b)
# @numba.njit
# def make_cls2(a,b): return Location2(a,b)
@numba.njit
def make_atom2(a,b): return (b << 16) | a

num = 10_000_000
a, b = 25, 25
result = []
result.append(("tup", run_test(make_tuple2, num, a, b)))
result.append(("atm", run_test(make_atom2, num, a, b)))
result.append(("arr", run_test(make_arr2, num, a, b)))
result.append(("arr", run_test(make_arr3, num, a, b)))
result.append(("dir", run_test(make_dir2, num, a, b)))
result.append(("pos", run_test(make_pos2, num, a, b)))
# result.append(("loc", run_test(make_loc2, num, a, b)))
# result.append(("cls", run_test(make_cls2, num, a, b)))
print_results(result)

tup: 4.4192 seconds (base)
atm: 3.9663 seconds, Speed relative to tup: 1.11x
arr: 7.5524 seconds, Speed relative to tup: 0.59x
arr: 17.8654 seconds, Speed relative to tup: 0.25x
dir: 4.0254 seconds, Speed relative to tup: 1.10x
pos: 18.4002 seconds, Speed relative to tup: 0.24x


In [49]:
@numba.njit
def get_row_tuple2(val): return val[0]
@numba.njit
def get_row_arr2(val): return val[0]
@numba.njit
def get_row_dir2(val): return val[0]
@numba.njit
def get_row_atom2(val): return val & 0xFFFF
@numba.njit
def get_col_atom2(val): return val >> 16 & 0xFFFF
@numba.njit
def get_bth_atom2(val): return val & 0xFFFF, val >> 16 & 0xFFFF

num = 10_000_000
a, b = 25, 25
t1 = (a,b)
t2 = np.array([a,b])
# t4 = Pos(a,b)
# t5 = Location(a,b)
# t6 = Location2(a,b)
t7 = (b << 16) | a

result = []
result.append(("tup    ", run_test(get_row_tuple2, num, t1)))
result.append(("atm_row", run_test(get_row_atom2, num, t7)))
result.append(("atm_col", run_test(get_col_atom2, num, t7)))
result.append(("atm_col", run_test(get_bth_atom2, num, t7)))
result.append(("arr    ", run_test(get_row_arr2, num, t2)))
# result.append(("dir", run_test(get_row_dir, num, a, b)))
# result.append(("pos    ", run_test(get_row_pos, num, t4)))
# result.append(("loc    ", run_test(get_row_loc, num, t5)))
# result.append(("cls    ", run_test(get_row_cls, num, t6)))
print_results(result)

tup    : 4.6809 seconds (base)
atm_row: 3.3257 seconds, Speed relative to tup    : 1.41x
atm_col: 3.3031 seconds, Speed relative to tup    : 1.42x
atm_col: 3.7431 seconds, Speed relative to tup    : 1.25x
arr    : 5.8834 seconds, Speed relative to tup    : 0.80x


In [70]:
def test_enum(a): return atom.AtomType(a)
def test_enum1(a, b): return a == b
def test_enum2(a, b): return a == b.value

num = 10_000_000
a, b = 1, atom.AtomType(1)
result = []
result.append(("num   ", run_test(test_enum1, num, a, a)))
result.append(("create", run_test(test_enum, num, a)))
result.append(("value ", run_test(test_enum2, num, a, b)))
result.append(("enum  ", run_test(test_enum1, num, a, b)))
print_results(result)

True
False
True
True
num   : 2.2308 seconds (base)
create: 8.5183 seconds, Speed relative to num   : 0.26x
value : 4.8952 seconds, Speed relative to num   : 0.46x
enum  : 2.1508 seconds, Speed relative to num   : 1.04x


In [77]:
print(Pos(25,24) == (25,24))
print(Location(25,24) == (25,24))
print(atom.AtomType(1) == 1)
print(atom.AtomType(1) == atom.AtomType(1))
print(atom.AtomType(1).value == 1)

True
False
False
True
True


In [84]:
def bit_and(a,b): return a&b
def bit_or(a,b): return a|b
def bit_xor(a,b): return a^b
def bit_sl(a,b): return a<<b
def bit_sr(a,b): return a>>b


num = 10_000_000
a, b = 22, 22
result = []
result.append(("and", run_test(bit_and, num, a, b)))
result.append(("or ", run_test(bit_or , num, a, b)))
result.append(("xor", run_test(bit_xor, num, a, b)))
result.append(("sl ", run_test(bit_sl , num, a, b)))
result.append(("sr ", run_test(bit_sr , num, a, b)))
print("a, b:",22,22)
print_results(result)
print()

a, b = 22, 0
result = []
result.append(("and", run_test(bit_and, num, a, b)))
result.append(("or ", run_test(bit_or , num, a, b)))
result.append(("xor", run_test(bit_xor, num, a, b)))
result.append(("sl ", run_test(bit_sl , num, a, b)))
result.append(("sr ", run_test(bit_sr , num, a, b)))
print("a, b:",22,0)
print_results(result)
print()

a, b = 0, 22
result = []
result.append(("and", run_test(bit_and, num, a, b)))
result.append(("or ", run_test(bit_or , num, a, b)))
result.append(("xor", run_test(bit_xor, num, a, b)))
result.append(("sl ", run_test(bit_sl , num, a, b)))
result.append(("sr ", run_test(bit_sr , num, a, b)))
print("a, b:",0,22)
print_results(result)
print()


a, b: 22 22
and: 2.9135 seconds (base)
or : 2.8695 seconds, Speed relative to and: 1.02x
xor: 3.0318 seconds, Speed relative to and: 0.96x
sl : 2.9141 seconds, Speed relative to and: 1.00x
sr : 2.9260 seconds, Speed relative to and: 1.00x

a, b: 22 0
and: 2.6007 seconds (base)
or : 2.3644 seconds, Speed relative to and: 1.10x
xor: 2.7448 seconds, Speed relative to and: 0.95x
sl : 2.9361 seconds, Speed relative to and: 0.89x
sr : 2.9558 seconds, Speed relative to and: 0.88x

a, b: 0 22
and: 2.3733 seconds (base)
or : 2.4649 seconds, Speed relative to and: 0.96x
xor: 2.3350 seconds, Speed relative to and: 1.02x
sl : 2.2245 seconds, Speed relative to and: 1.07x
sr : 2.3101 seconds, Speed relative to and: 1.03x



In [85]:
def cmp_bit(a, b): return bool(a & b)
def cmp_bit2(a, b): return bool(a | b)
def cmp_bit3(a, b): return bool(a ^ b)
def cmp_bool(a, b): return a == b
def cmp_bool2(a, b): return a and b
def cmp_bool3(a, b): return a or b

num = 10_000_000
a, b = 22, 22
result = []
result.append(("land", run_test(cmp_bit  , num, a, b)))
result.append(("lor", run_test(cmp_bit2 , num, a, b)))
result.append(("xor", run_test(cmp_bit3 , num, a, b)))
result.append(("== ", run_test(cmp_bool , num, a, b)))
result.append(("and", run_test(cmp_bool2, num, a, b)))
result.append(("or ", run_test(cmp_bool3, num, a, b)))
print_results(result)

land: 2.8127 seconds (base)
lor: 3.2131 seconds, Speed relative to land: 0.88x
xor: 2.9866 seconds, Speed relative to land: 0.94x
== : 2.0931 seconds, Speed relative to land: 1.34x
and: 2.2071 seconds, Speed relative to land: 1.27x
or : 2.3117 seconds, Speed relative to land: 1.22x


In [90]:
a = [None] * 10
a[0] = 1
a

[1, None, None, None, None, None, None, None, None, None]

In [96]:
class Color(Enum):
  red = 0
  blue = 1
  green = 2

class Color2:
  red = Color.red
  blue = Color.blue
  green = Color.green

def f(e):
  for _ in range(1000):
    e.red
    e.blue
    e.green

print(timeit.timeit('f(Color)', number=10000, globals=globals()))
print(timeit.timeit('f(Color2)', number=10000, globals=globals()))


2.894084099913016
2.9426833000034094


In [46]:
def make_int(a:int) -> int: return a
def make_enum(a:int) -> Color: return Color(a)
def cmp_int(a:int,b:int) -> bool: return a == b
def cmp_enum(a:Color,b:Color) -> bool: return a == b
def cmp_enum_int(a:Color,b:int) -> bool: return a.value == b
def get_red() -> Color: return Color.red
def get_red_val() -> int: return Color.red.value

num = 10_000_000
a, b, c, d = 1, 1, Color(1), Color(1)
result = []
result.append(("mint ", run_test(make_int  , num, a)))
result.append(("menum", run_test(make_enum , num, a)))
result.append(("cint ", run_test(cmp_int, num, a, b)))
result.append(("cenum", run_test(cmp_enum, num, c, d)))
result.append(("cenin", run_test(cmp_enum_int, num, c, b)))
result.append(("getre", run_test(get_red, num)))
result.append(("reval", run_test(get_red_val, num)))
print_results(result)

NameError: name 'Color' is not defined

In [55]:
a = [i for i in range(100)]
print(len(a[:None]) == len(a[:100]) == 100)

def t1(a, b=None): return a[:b]
num = 10_000_000
result = []
result.append(("t1_a", run_test(t1, num, a)))
result.append(("t1_b", run_test(t1, num, a, 100)))
result.append(("t2", run_test(t1, num, a, 0)))
print_results(result)


True
t1_a: 11.7175 seconds (base)
t1_b: 11.9775 seconds, Speed relative to t1_a: 0.98x
t2: 4.2991 seconds, Speed relative to t1_a: 2.73x
